# Initial Data Reading and Cleaning

In [109]:
import pandas as pd
import numpy as np

df = pd.read_csv('./ingredients_refined.csv')

df = df.drop_duplicates(subset=['ingredient'])

ingredients = np.array(df['ingredient'].values)
price = np.array(df['price'].values)
metric = np.array(df['metric'].values)

# Building the amount matrix

In [110]:
import json

with open('./train.json', 'r') as f:
    recipes = json.load(f)

matrix = pd.DataFrame(columns=ingredients, index = [recipe['id'] for recipe in recipes])

for recipe in recipes:
    for ingredient in recipe["ingredients"]:
        for column in matrix.columns:
            if ingredient in column:  # Check if ingredient is a substring of the column
                matrix.loc[recipe["id"], column] = 1

matrix = matrix.fillna(0)

/var/folders/l7/ywb8q5r53j33j1lqzbzjyshm0000gn/T/ipykernel_31755/3077128867.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix = matrix.fillna(0)


In [111]:
matrix

,lemon,cream,yogurt,vanilla,shrimp,cilantro,jalapeno,parsley,vinegar,tortillas,...,black pepper,chili,coriander,jalapenos,mango,oil,olives,onions,peppers,pork loin
10259,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,1,0
25693,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
20130,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,1,1,0
6602,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42779,0,0,0,0,1,1,0,1,1,0,...,1,0,0,0,0,1,0,0,1,0
16903,0,0,0,0,0,1,0,0,0,1,...,1,1,0,1,0,1,0,1,1,0
45887,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2698,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
41995,0,0,0,0,0,1,0,0,0,0,...,1,1,1,0,0,1,0,1,0,0
31908,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


# Modeling

In [116]:
import gamspy as gp
import numpy as np

cont = gp.Container()
I = cont.addSet('I', records = [ingredients[idx] for idx in range(len(ingredients))])
R = cont.addSet('R', records = [recipe['id'] for recipe in recipes])

In [125]:
A = cont.addParameter('A', domain = [R, I], description = 'Amount of ingredient i required by recipe r' , records = np.array(matrix))

In [127]:
A.pivot()

,red pepper,american cheese,cheddar cheese,iceburg lettuce,pepper,bean,romaine lettuce,garlic powder,onion powder,cumin,...,chicken stock,parmesan,spaghetti,bacon,rice,basil,pork loin,olives,paprika,mango
10259,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25693,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20130,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42779,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16903,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45887,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2698,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31908,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
